In [1]:
!pip install pandas sagemaker

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sagemaker
from sagemaker import get_execution_role

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/25/25 00:51:49] INFO     Found credentials from IAM Role:                                   ]8;id=829585;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=865629;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
role = get_execution_role()
session = sagemaker.Session()
bucket = session.default_bucket()  # SageMaker will create an S3 bucket for you


[05/25/25 00:51:54] INFO     Found credentials from IAM Role:                                   ]8;id=31730;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=853471;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[05/25/25 00:51:55] INFO     Found credentials from IAM Role:                                   ]8;id=847736;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=974646;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [4]:
red_wine = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')
white_wine = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=';')


In [5]:
red_wine['type'] = 0
white_wine['type'] = 1
data = pd.concat([red_wine, white_wine])


In [6]:
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

In [8]:
train_data.to_csv('train.csv', index=False, header=False)
test_data.to_csv('test.csv', index=False, header=False)

In [9]:
s3_train_path = session.upload_data('train.csv', bucket=bucket, key_prefix='wine/train')
s3_test_path = session.upload_data('test.csv', bucket=bucket, key_prefix='wine/test')


In [10]:
print(f"Training data uploaded to: {s3_train_path}")
print(f"Test data uploaded to: {s3_test_path}")

Training data uploaded to: s3://sagemaker-us-east-2-056614919553/wine/train/train.csv
Test data uploaded to: s3://sagemaker-us-east-2-056614919553/wine/test/test.csv


In [11]:
from sagemaker.amazon.amazon_estimator import get_image_uri


In [12]:
container = sagemaker.image_uris.retrieve('xgboost', session.boto_region_name, '1.2-1')


[05/25/25 00:54:40] INFO     Ignoring unnecessary instance type: None.                            ]8;id=308373;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=514082;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

In [13]:
xgb = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{bucket}/wine/output',
    sagemaker_session=session
)


[05/25/25 00:54:59] INFO     Found credentials from IAM Role:                                   ]8;id=729946;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=179543;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [14]:
xgb.set_hyperparameters(
    objective='reg:squarederror',  # For regression
    num_round=100,                # Number of training rounds
    max_depth=5                   # Tree depth
)


In [15]:
xgb.fit({'train': s3_train_path, 'test': s3_test_path})

[05/25/25 00:55:27] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=405964;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=917416;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=299938;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=24703;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-05-25-00-55-27-218                                             

2025-05-25 00:55:27 Starting - Starting the training job......
2025-05-25 00:56:18 Starting - Preparing the instances for training...
2025-05-25 00:56:38 Downloading - Downloading input data...
2025-05-25 00:57:23 Downloading - Downloading the training image......
2025-05-25 00:58:29 Training - Training image download completed. Training in progress.
2025-05-25 00:58:29 Uploading - Uploading generated training model[2025-05-25 00:58:23.444 ip-10-0-82-210.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
ERROR:sagemaker-containers:Reporting training FAILURE
ERROR:sagemaker-containers:framework 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 xgb.fit({'train': s3_train_path, 'test': s3_test_path})                                      │
│   2                                                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:167 in wrapper                                                                       │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:138 in wrapper                                                                       │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_c │
│ ontext.py:346 in wrapper                                                                         │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-p

In [ ]:
Deploy Model

In [ ]:
predictor = xgb.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name='wine-quality-predictor'
)


In [ ]:
# Test with sample red wine data

In [ ]:
import numpy as np
sample = [7.4, 0.7, 0.0, 1.9, 0.076, 11, 34, 0.9978, 3.51, 0.56, 9.4, 0]  # Last 0 = red wine
prediction = predictor.predict(np.array([sample]))
print(f"Predicted wine quality: {prediction}")

In [ ]:
Clean Up

In [ ]:
predictor.delete_endpoint()